In [21]:
import os
from pathlib import Path
from dotenv import load_dotenv

In [22]:
load_dotenv(Path("Environment Variables.env"))

True

In [23]:
OpenAI_API_key = os.getenv('OPENAI_API_KEY')
langchain_tracing_V2 = os.getenv("LANGCHAIN_TRACING_V2")
langchain_API_key = os.getenv("LANGCHAIN_API_KEY")

In [24]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(openai_api_key=OpenAI_API_key)

In [27]:
llm.invoke("how can langsmith help with testing?")

AIMessage(content='Langsmith can help with testing by providing automated testing tools, such as test frameworks and test scripts, that can be used to automate the testing process. It can also help with generating test data, running tests in parallel, and integrating with continuous integration tools to streamline the testing process. Additionally, Langsmith can provide insights and analytics on test results, helping to identify and address any issues or bugs in the code.')

In [30]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

prompt

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are world class technical documentation writer.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])

In [31]:
chain = prompt | llm 
chain

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are world class technical documentation writer.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000025D421D8C10>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000025D421DE160>, openai_api_key=SecretStr('**********'), openai_proxy='')

In [32]:
chain.invoke({"input": "how can langsmith help with testing?"})

AIMessage(content='Langsmith can help with testing in several ways:\n\n1. Test case generation: Langsmith can generate test cases automatically based on the specifications and requirements provided. This can help in quickly creating a comprehensive set of test cases to cover different scenarios and edge cases.\n\n2. Test automation: Langsmith can be used to automate the execution of test cases, saving time and effort in manual testing. It can run the test cases repeatedly and consistently, ensuring that the application behaves as expected across different builds and environments.\n\n3. Code analysis: Langsmith can perform static code analysis to identify potential issues and bugs in the code before testing. This can help in improving code quality and reducing the number of defects found during testing.\n\n4. Performance testing: Langsmith can simulate the behavior of multiple users accessing the application simultaneously to test its performance under different load conditions. This ca